In [1]:
import requests
import csv
import os
import pandas as pd
import json
import re
from datetime import datetime
today_string = datetime.now().strftime('%Y-%m-%d')

In [4]:
input_path = "C:\\Users\\khan32\\Documents\\factgrid_recon"

In [6]:
input_file = f"WIAG-Domherren-DB-Lebensdaten_2024-05-29.csv"
input_path_file = os.path.join(input_path, input_file)
persons_df = pd.read_csv(input_path_file, sep=';')
print(len(persons_df))
persons_df.head()

15584


,id,corpus,givenname,prefix,familyname,displayname,date_of_birth,date_of_death,biographical_dates,summary_offices,GND_ID,GSN,FactGrid_ID,Wikidata_ID,Wikipedia
0,WIAG-Pers-EPISCGatz-03848-001,epc,Ulrich,NaN,NaN,Ulrich,NaN,973,+ 973,"Bischof Augsburg 923-973, Domherr Domstift Aug...",118625284,059-01621-001,Q653546,Q714866,Ulrich_von_Augsburg
1,WIAG-Pers-EPISCGatz-03858-001,epc,Siegfried,NaN,NaN,Siegfried,NaN,1096,+ 1096,"Ernannter Bischof Augsburg 1077-1096, Domherr ...",13805746X,050-06892-001,Q653421,Q64763639,NaN
2,WIAG-Pers-EPISCGatz-04027-001,epc,Norbert,NaN,NaN,Norbert,NaN,NaN,~ 1079/1080,Bischof Chur 1079/1080-1088,NaN,710-00332-001,Q653192,Q35136089,Norbert_von_Hohenwart
3,WIAG-Pers-CANON-91597-001,dreg-can,Sizo,NaN,NaN,Sizo,NaN,NaN,~ um 1099,Domkustos Domstift Augsburg um 1099,NaN,710-00420-001,Q700910,NaN,NaN
4,WIAG-Pers-EPISCGatz-02518-001,epc,Udalschalk,NaN,NaN,Udalschalk,NaN,1202,+ 1202,"Bischof Augsburg 1184-1202, Dompropst Domstift...",138018383,059-01616-001,Q649639,Q68911884,NaN


In [7]:
url = 'https://database.factgrid.de/sparql'
query = (
"""
SELECT ?person ?wiag WHERE {
  ?person wdt:P601 ?wiag.
}
"""
)

r = requests.get(url, params={'query': query}, headers={"Accept": "application/json"})
data = r.json()
wiag_ids_df = pd.json_normalize(data['results']['bindings'])

len(wiag_ids_df)

11750

In [10]:
# extract out q id
def extract_qid(df, column):
    df[column] = df[column].map(lambda x: x.strip('https://database.factgrid.de/entity/'))
 
#factgrid_df['item.value'] = factgrid_df['item.value'].map(lambda x: x.strip('https://database.factgrid.de/entity/'))

# drop irrelevant columns
def drop_type_columns(df):
    df.drop(columns=[column for column in df.columns if column.endswith('type')], inplace=True)
    df.drop(columns=[column for column in df.columns if column.endswith('xml:lang')], inplace=True)

In [13]:
drop_type_columns(wiag_ids_df)
extract_qid(wiag_ids_df, 'person.value')
wiag_ids_df

,person.value,wiag.value
0,Q16188,WIAG-Pers-EPISCGatz-10475-001
1,Q16194,WIAG-Pers-EPISCGatz-10816-001
2,Q153179,WIAG-Inst-DIOCGatz-001-001
3,Q153216,WIAG-Inst-DIOCGatz-002-001
4,Q153217,WIAG-Inst-DIOCGatz-003-001
...,...,...
11745,Q883782,WIAG-Pers-CANON-10517-001
11746,Q883831,WIAG-Pers-CANON-10684-001
11747,Q883783,WIAG-Pers-CANON-14951-001
11748,Q883731,WIAG-Pers-CANON-15040-001
